In [2]:
# to deal with an issue with importing opencv 
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [3]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
import torch
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
from datetime import datetime
import numpy as np
import cv2
import os
import sys
import random

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.utils.visualizer import Visualizer
from detectron2.engine import DefaultTrainer


In [4]:
register_coco_instances("training_set", {}, "./datasets/coco_annts/pascal_train2007.json", 
                        "./datasets/voc_trainval/VOC2007/JPEGImages/")

In [5]:
train_dics = DatasetCatalog.get("training_set")
train_metadata = MetadataCatalog.get("training_set")

[01/10 01:52:40 d2.data.datasets.coco]: Loaded 2501 images in COCO format from ./datasets/coco_annts/pascal_train2007.json


In [6]:
def show_img(vis, msg="", is_vis =True):
    if is_vis:
        cv2.imshow(msg, vis.get_image()[:, :, ::-1])
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else: 
        cv2.imshow(msg, vis)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

In [7]:
for d in random.sample(train_dics, 3):
    img = cv2.imread(d["file_name"])
    visualizer = Visualizer(img[:, :, ::-1], metadata=train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    show_img(vis)

In [ ]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can either use the https://dl.fbaipublicfiles.... url, or use the detectron2:// shorthand
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
predictor = DefaultPredictor(cfg)
im = cv2.imread(train_dics[0]["file_name"])
show_img(im, is_vis=False)
outputs = predictor(im)

In [ ]:
# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
show_img(v)

In [8]:
torch.cuda.empty_cache()

In [9]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
assert(cfg.MODEL.MASK_ON)
cfg.DATASETS.TRAIN = ("training_set",)
cfg.DATASETS.TEST = ()
# cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # Let training initialize from model zoo
# cfg.SOLVER.IMS_PER_BATCH = 2
# cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
# cfg.SOLVER.STEPS = ('2100', '2100')
cfg.SOLVER.MAX_ITER = 1000    # 300 iterations seems good enough for this toy dataset; you may need to train longer for a practical dataset
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 200   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 20  

now = datetime.now()

cfg.OUTPUT_DIR = './pascal_inseg' + str(now.year) + str(now.month) + str(now.day) + str(now.hour)
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[01/10 01:53:09 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[01/10 01:53:09 d2.data.datasets.coco]: Loaded 2501 images in COCO format from ./datasets/coco_annts/pascal_train2007.json
[01/10 01:53:09 d2.data.build]: Removed 0 images with no usable annotations. 2501 images left.
[01/10 01:53:09 d2.data.build]: Distribution of instances among all 20 categories:
|  category   | #instances   |  category   | #instances   |  category  | #instances   |
|:-----------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
|  aeroplane  | 151          |   bicycle   | 176          |    bird    | 243          |
|    boat     | 140          |   bottle    | 253          |    bus     | 115          |
|     car     | 625          |     cat     | 186          |   chair    | 400          |
|     cow     | 136          | diningtable | 103          |    dog     | 253          |
|    horse    | 182          |  motorbike  | 167          |   person   | 2358         |
| pottedplant | 248          |    sheep    | 130          |    sofa    | 124       

'roi_heads.box_predictor.cls_score.weight' has shape (81, 1024) in the checkpoint but (21, 1024) in the model! Skipped.
'roi_heads.box_predictor.cls_score.bias' has shape (81,) in the checkpoint but (21,) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.weight' has shape (320, 1024) in the checkpoint but (80, 1024) in the model! Skipped.
'roi_heads.box_predictor.bbox_pred.bias' has shape (320,) in the checkpoint but (80,) in the model! Skipped.
'roi_heads.mask_head.predictor.weight' has shape (80, 256, 1, 1) in the checkpoint but (20, 256, 1, 1) in the model! Skipped.
'roi_heads.mask_head.predictor.bias' has shape (80,) in the checkpoint but (20,) in the model! Skipped.


[01/10 01:53:09 d2.engine.train_loop]: Starting training from iteration 0
[01/10 01:53:11 d2.engine.hooks]: Total training time: 0:00:01 (0:00:00 on hooks)


RuntimeError: CUDA out of memory. Tried to allocate 1000.00 MiB (GPU 0; 10.91 GiB total capacity; 9.60 GiB already allocated; 318.62 MiB free; 264.30 MiB cached)

In [ ]:
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE

In [ ]:
%load_ext tensorboard
%tensorboard --logdir cfg.OUTPUT_DIR

In [ ]:
register_coco_instances("test_set", {}, "./datasets/coco_annts/pascal_test2007.json", 
                        "./datasets/voc_test/VOC2007/JPEGImages/")

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7   # set the testing threshold for this model
cfg.DATASETS.TEST = ("train_set", )
predictor = DefaultPredictor(cfg)

In [ ]:
from detectron2.utils.visualizer import ColorMode
dataset_dicts = train_dics = DatasetCatalog.get("test_set")
balloon_metadata = MetadataCatalog.get("test_set")
for d in random.sample(dataset_dicts, 3):    
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1],
                   metadata=balloon_metadata, 
                   scale=0.8, 
                   instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    show_img(v)

In [ ]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("test_set", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "test_set")
inference_on_dataset(trainer.model, val_loader, evaluator)
# another equivalent way is to use trainer.test